<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>


# Creating and Provisioning Storage with Python

## Azure Actions Covered

* Creating storage resources


In this lecture, we're going to take a look at how to create storage resources via the Python SDK.

To begin, we'll need to import our usual libraries as well as any useful environment variables (e.g. `AZURE_SUBSCRIPTION_ID`). We'll add our new storage import as well.

In [1]:
# For generating storage account name
import random

from azure.identity import AzureCliCredential
from azure.mgmt.resource import ResourceManagementClient
# New import
from azure.mgmt.storage import StorageManagementClient

from settings import AZURE_SUBSCRIPTION_ID, DEFAULT_LOCATION, DEFAULT_RESOURCE_GROUP

We need to instantiate our Azure credential as well as our `ResourceManagementClient()` object.

In [2]:
credential = AzureCliCredential()
resource_client = ResourceManagementClient(credential, AZURE_SUBSCRIPTION_ID)

We'll need to create our Storage objects under a resource group, so let's create a default Resource Group in a default Azure location if one doesn't exist yet.

In [4]:
if not resource_client.resource_groups.get(DEFAULT_RESOURCE_GROUP):
    rg_result = resource_client.resource_groups.create_or_update(
        DEFAULT_RESOURCE_GROUP,
        {'location': DEFAULT_LOCATION}
    )

We can instantiate a `StorageManagementClient()` object using the same pattern as with `ResourceManagementClient()`.

In [3]:
storage_client = StorageManagementClient(credential, AZURE_SUBSCRIPTION_ID)

As we saw in the previous lecture, storage accounts must have unique names since they're used in our URLs. We'll use a `while` loop to try out variations on a base name for our account until we find one that works.
* First, use `storage_client.storage_accounts.check_name_availability()` to check whether the generated name is available.
* If the name isn't available, generate a new one.
* If the name is available, use `begin_create()` to create the account. You can specify options like we did in the CLI.
    * `location` - Azure location
    * `kind` - Indicates the type of storage account, such as `StorageV2`, `FileStorage`, or `BlobStorage`.
    * `sku` - Dictionary of SKU options for storage account, such as the name of the SKU, including `Standard_LRS`, `Standard_GRS`, or `Standard_ZRS`.
    * `tags` - Dictionary of key-value pairs for the resource.

In [15]:
STORAGE_ACCOUNT_BASE = 'benbstorage'
while True:
    account_name = f'{STORAGE_ACCOUNT_BASE}{random.randint(1,1000)}'
    if not storage_client.storage_accounts.check_name_availability({'name':account_name}).name_available:
        print(f'Account name {account_name} not available, trying again')
    else:
        print(f'Account name {account_name} available, creating account')
        poller =  storage_client.storage_accounts.begin_create(
            DEFAULT_RESOURCE_GROUP,
            account_name,
            {
                'location': DEFAULT_LOCATION,
                'kind': 'StorageV2',
                'sku': {'name': 'Standard_LRS'}
            }
        )
        poller_result = poller.result()
        print(f'{poller_result.name} has been provisioned')
        break

Account name benbstorage226 available, creating account
benbstorage226 has been provisioned


We can use our storage client to create a blob container under our storage account using `blob_containers.create()`.

In [17]:
container = storage_client.blob_containers.create(
    DEFAULT_RESOURCE_GROUP,  # Name of resource group
    account_name,  # Name of storage account
    'bens-blob-container',  # Name of container
    {}  # Properties of the blob container
)

Let's validate that our container was created.

In [19]:
container.name

'bens-blob-container'

To-do:
* Create file share
* Create queue
* Create table